In [95]:
import requests
import pandas as pd
import dill
import matplotlib.pyplot as plt

from requests.exceptions import RequestException

econ_df = pd.read_pickle('econ_df.pkl')

## Summary / Notes

This notebook ingests and processes data from the Colorado Dept. of Revenue and the US Census Bureau, with the results saved in the dataframe <b>'econ_df.pkl'</b>.



### Links:
Info on dispensaries by county found here: https://cdor.colorado.gov/data-and-reports/marijuana-data/marijuana-sales-reports <br><br>
Small Area Income and Poverty Estimates
https://api.census.gov/data/timeseries/poverty/saipe.html

US Census American Community Survey has good data, but only begins at 2005.  May revisit later.
https://www.census.gov/data/developers/data-sets/acs-1year.html <br>
Titled "American Community Survey: 1-Year Estimates: Detailed Tables 1-Year" in list of APIs (https://api.census.gov/data.html)

Unemployment from USDA ERS: https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/

## Marijuana Sales by County

In [86]:
# Initial import
weed_df = pd.read_csv('Marijuana_Sales_By_County_2014_To_Date_Report.xlsx - County Report - edit.csv')

# Convert relevant columns to numeric values
weed_df['Medical Marijuana Sales'] = weed_df['Medical Marijuana Sales'].str.replace(r'[^0-9.]', '', regex = True)
weed_df['Medical Marijuana Sales'] = pd.to_numeric(weed_df['Medical Marijuana Sales'], errors='coerce')

weed_df['Retail Marijuana Sales'] = weed_df['Retail Marijuana Sales'].str.replace(r'[^0-9.]', '', regex = True)
weed_df['Retail Marijuana Sales'] = pd.to_numeric(weed_df['Retail Marijuana Sales'], errors='coerce')

# Group by year and county
weedyear_df = weed_df.groupby(['Year', 'County']).sum().reset_index().drop(columns = 'Month')
weedyear_df = weedyear_df.rename(columns = {'Year': 'year', 'County': 'county'})
weedyear_df['county'] = weedyear_df['county'].str.upper().str.replace('TOTAL', 'COLORADO')

In [110]:
weedyear_df

,year,county,Medical Marijuana Sales,Retail Marijuana Sales
0,2014,COLORADO,380284040.0,303239699.0
1,2014,ADAMS,5781889.0,2749908.0
2,2014,ALAMOSA,0.0,0.0
3,2014,ARAPAHOE,16861284.0,2221928.0
4,2014,ARCHULETA,0.0,0.0
...,...,...,...,...
422,2023,SAN JUAN,0.0,24753.0
423,2023,SAN MIGUEL,0.0,1618613.0
424,2023,SEDGWICK,0.0,3567541.0
425,2023,SUMMIT,0.0,8169430.0


## American Community Survey

In [118]:
# https://api.census.gov/data/2005/acs/acs1?get=NAME,B01001_001E ,B25105_001M,B25105_001MA'
url = 'https://api.census.gov/data/2016/acs/acs1'
params = {
    'get': 'NAME,B25105_001EA,B25105_001M,B25105_001MA',
    'for': 'county:*',
    'in': 'state:08',
}

response = requests.get(url, params=params)

In [119]:
response.json()

[['NAME', 'B25105_001EA', 'B25105_001M', 'B25105_001MA', 'state', 'county'],
 ['Denver County, Colorado', None, '28', None, '08', '031'],
 ['Jefferson County, Colorado', None, '25', None, '08', '059'],
 ['Weld County, Colorado', None, '42', None, '08', '123'],
 ['Douglas County, Colorado', None, '37', None, '08', '035'],
 ['Boulder County, Colorado', None, '35', None, '08', '013'],
 ['Pueblo County, Colorado', None, '36', None, '08', '101'],
 ['Mesa County, Colorado', None, '43', None, '08', '077'],
 ['El Paso County, Colorado', None, '19', None, '08', '041'],
 ['Arapahoe County, Colorado', None, '24', None, '08', '005'],
 ['Broomfield County, Colorado', None, '60', None, '08', '014'],
 ['Adams County, Colorado', None, '21', None, '08', '001'],
 ['Larimer County, Colorado', None, '36', None, '08', '069']]

## Small Area Income and Poverty Estimates (SAIPE)

Colorado state code: 08 <br>
County codes: ['0', '1', '3', '5', '7', '9', '11', '13', '14', '15', '17', 
                '19', '21', '23', '25', '27', '29', '31', '33', '35', '37', 
                '39', '41', '43', '45', '47', '49', '51', '53', '55', '57', 
                '59', '61', '63', '65', '67', '69', '71', '73', '75', '77', 
                '79', '81', '83', '85', '87', '89', '91', '93', '95', '97', 
                '99', '101', '103', '105', '107', '109', '111', '113', '115',
                '117', '119', '121', '123', '125']
                

API documentation says Small Area Income and Poverty Estimates only goes to 2016, but requests below work until 2021

Complete list of variables available at: https://api.census.gov/data/timeseries/poverty/saipe/variables.html

Variables obtained: <br>
Median Household Income Estimate: SAEMHI_PT <br>
All ages in Poverty, Count Estimate: SAEPOVALL_PT <br>
All ages in Poverty, Rate Estimate: SAEPOVRTALL_PT <br>
Ages 5-17 in Families in Poverty, Count Estimate: SAEPOV5_17R_PT <br>
Ages 5-17 in Families in Poverty, Rate Estimate: SAEPOVRT5_17R_PT

In [4]:
# List of variables available at: https://api.census.gov/data/timeseries/poverty/saipe/variables.html

url = 'https://api.census.gov/data/timeseries/poverty/saipe'
params = {
    'get': 'NAME,SAEMHI_PT,SAEPOVALL_PT,SAEPOVRTALL_PT,SAEPOV5_17R_PT,SAEPOVRT5_17R_PT',
    'for': 'county:*',
    'in': 'state:08',
    'time': 'from 2000 to 2021'
}

response = requests.get(url, params=params).json()

In [64]:
# Missing one year for Broomfield County
saipe_df = pd.DataFrame(response[1:], columns = response[0])

In [65]:
saipe_df = saipe_df.drop(['state', 'county'], axis = 1) \
                   .rename(columns = {'NAME': 'county', 'time': 'year', 'SAEMHI_PT': 'median household income',
                                      'SAEPOVALL_PT': 'all poverty count', 'SAEPOVRTALL_PT': 'all poverty rate',
                                      'SAEPOV5_17R_PT': 'age5-17 poverty count', 'SAEPOVRT5_17R_PT': 'age5-17 poverty rate'}) \

saipe_df['county'] = saipe_df['county'].str.replace(' County', '', regex=False).str.upper()



In [70]:
cols = ['median household income', 'all poverty count', 'all poverty rate', 
       'age5-17 poverty count', 'age5-17 poverty rate', 'year']

saipe_df[cols] = saipe_df[cols].apply(pd.to_numeric, errors='coerce')

In [71]:
saipe_df

,county,median household income,all poverty count,all poverty rate,age5-17 poverty count,age5-17 poverty rate,year
0,ADAMS,48307,33377,8.9,8806,12.0,2000
1,ALAMOSA,30538,2851,20.3,706,24.1,2000
2,ARAPAHOE,55359,29023,5.8,6514,6.8,2000
3,ARCHULETA,38241,1379,13.2,377,18.9,2000
4,BACA,29274,773,17.5,188,23.5,2000
...,...,...,...,...,...,...,...
1402,SUMMIT,90297,2053,6.7,225,6.4,2021
1403,TELLER,76715,1968,7.9,388,12.2,2021
1404,WASHINGTON,57522,544,11.6,128,15.2,2021
1405,WELD,84826,32543,9.8,6942,11.0,2021


## Population Estimates

In [ ]:
# US Census API for population estimates changed year to year and was hard to 
# navigate.  Found Excel spreadsheets at these links:
# https://www.census.gov/data/tables/time-series/demo/popest/intercensal-2000-2010-counties.html
# https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html
# https://www.census.gov/data/datasets/time-series/demo/popest/2020s-counties-total.html

In [58]:
pop2000s = pd.read_csv('population data/co-est00int-01-08-edit.csv', header=1)
pop2000s = pop2000s.rename(columns = {'Unnamed: 0': 'region'})
pop2000s['region'] = pop2000s['region'].str.replace('.', '', regex=False).str.replace(' County', '').str.upper()

pop2010s = pd.read_csv('population data/co-est2019-annres-08-edit.csv', header=1)
pop2010s = pop2010s.rename(columns = {'Unnamed: 0': 'region'})
pop2010s['region'] = pop2010s['region'].str.replace('.', '', regex=False).str.replace(' County, Colorado', '').str.upper()

pop2020s = pd.read_csv('population data/co-est2022-pop-08-edit.csv', header=1)
pop2020s = pop2020s.rename(columns = {'Unnamed: 0': 'region'})
pop2020s['region'] = pop2020s['region'].str.replace('.', '', regex=False).str.replace(' County, Colorado', '').str.upper()

countypop_df = pop2000s.merge(pop2010s, on = 'region')
countypop_df = countypop_df.merge(pop2020s, on = 'region')

In [59]:
# Create an empty list to store the reformatted rows
new_rows = []

# Iterate over each row in the original dataframe
for index, row in countypop_df.iterrows():
    region = row['region']
    # Iterate over each column except the 'region' column
    for year, population in list(row.items())[1:]:
        new_rows.append([region, year, population])

# Create a new dataframe using the reformatted rows
countypop_df = pd.DataFrame(new_rows, columns=['county', 'year', 'population'])

In [60]:
countypop_df['population'] = countypop_df['population'].str.replace(',', '')
countypop_df['population'] = pd.to_numeric(countypop_df['population'], errors = 'coerce')

countypop_df['year'] = pd.to_numeric(countypop_df['year'], errors = 'coerce')

## Unemployment Statistics

In [31]:
unemployment = pd.read_excel('population data/unemployment.xlsx', header=4)
unemployment = unemployment[unemployment['State'] == 'CO']

In [32]:
columns_to_remove = unemployment.filter(regex=r'^Employed_\d{4}$').columns
unemployment = unemployment.drop(columns=columns_to_remove)

unemployment = unemployment.drop(columns = ['FIPS_code', 'State', 'Urban_influence_code_2013', 
                                           'Rural_urban_continuum_code_2013', 'Metro_2013',
                                           'Median_Household_Income_2020', 'Med_HH_Income_Percent_of_State_Total_2020'])

In [93]:
new_columns = ['county', 'year', 'civilian_labor_force', 'unemployed', 'unemployment_rate']

# Create an empty dictionary to store the data
data_dict = {col: [] for col in new_columns}

for _, row in unemployment.iterrows():
    area_name = row['Area_name']
    
    # Iterate over the columns and extract the year and values
    for col_name, value in row.items():
        if col_name.startswith('Civilian_labor_force_'):
            year = col_name.split('_')[3]
            civilian_labor = value
            unemployed = row['Unemployed_' + year]
            unemployment_rate = row['Unemployment_rate_' + year]
            
            # Add the extracted values to the dictionary
            data_dict['county'].append(area_name)
            data_dict['year'].append(year)
            data_dict['civilian_labor_force'].append(civilian_labor)
            data_dict['unemployed'].append(unemployed)
            data_dict['unemployment_rate'].append(unemployment_rate)

# Create the DataFrame from the dictionary
unemployment_df = pd.DataFrame(data_dict)

In [94]:
unemployment_df['county'] = unemployment_df['county'].str.replace('Denver County/city, CO', 'DENVER')
unemployment_df['county'] = unemployment_df['county'].str.replace(' County, CO', '').str.upper()
unemployment_df['year'] = pd.to_numeric(unemployment_df['year'], errors = 'coerce')

## CPI Info

In [120]:
# from https://www.ssa.gov/oact/STATS/avgcpi.html
CPI = pd.read_csv('population data/CPIdata.csv')

In [124]:
CPIdict = dict(zip(CPI['Year'], CPI['Annual']))
CPIdict

{2000: 168.892,
 2001: 173.517,
 2002: 175.875,
 2003: 179.808,
 2004: 184.483,
 2005: 190.975,
 2006: 197.142,
 2007: 202.767,
 2008: 211.053,
 2009: 209.63,
 2010: 213.967,
 2011: 221.575,
 2012: 226.229,
 2013: 229.324,
 2014: 232.771,
 2015: 231.81,
 2016: 234.076,
 2017: 239.051,
 2018: 245.146,
 2019: 249.222,
 2020: 252.248,
 2021: 265.51}

## Merge Dataframes

In [125]:
econ_df = countypop_df.merge(saipe_df, on = ['year', 'county'], how = 'left')
econ_df = econ_df.merge(weedyear_df, on = ['year', 'county'], how = 'left')
econ_df['Medical Marijuana Sales'].fillna(0, inplace=True)
econ_df['Retail Marijuana Sales'].fillna(0, inplace=True)

econ_df = econ_df.merge(unemployment_df, on = ['year', 'county'], how = 'left')
econ_df['labor participation rate'] = econ_df['civilian_labor_force'] / econ_df['population']

In [125]:
econ_df.columns

Index(['county', 'year', 'population', 'median household income',
       'all poverty count', 'all poverty rate', 'age5-17 poverty count',
       'age5-17 poverty rate', 'Medical Marijuana Sales',
       'Retail Marijuana Sales', 'civilian_labor_force', 'unemployed',
       'unemployment_rate', 'labor participation rate'],
      dtype='object')

In [132]:
def adjust_income(row):
    try:
        cpi2021 = CPIdict.get(2021)
        year = row['year']
        cpi = CPIdict.get(year)  # Get the CPI value for the corresponding year
        income = row['median household income']
        adjusted_income = income / cpi * cpi2021
        return adjusted_income
    except:
        return None

In [141]:
econ_df['median_income_2021CPI'] = econ_df.apply(lambda row: adjust_income(row), axis=1)

In [146]:
econ_df.to_pickle('econ_df.pkl')